In [1]:
import pandas as pd
import numpy as np
import pygraphviz as pgv

In [2]:
frame = pd.read_csv('/Users/Lutz/Desktop/spiced_projects/discrete-dill-student-code/J3L/notebooks/lutz/supermarket.csv')

In [4]:
frame = frame.set_index('timestamp')
frame.index = pd.to_datetime(frame.index)
frame.head()

,customer_no,location,day
timestamp,,,
2019-09-02 07:03:00,2,dairy,Monday
2019-09-02 07:03:00,1,dairy,Monday
2019-09-02 07:04:00,3,dairy,Monday
2019-09-02 07:04:00,8,fruit,Monday
2019-09-02 07:04:00,4,dairy,Monday


In [5]:
frame_group = frame.groupby(['customer_no','day'])['location'].resample('1T').ffill()

In [6]:
frame_group.head(10)

customer_no  day       timestamp          
1            Friday    2019-09-06 07:00:00       dairy
                       2019-09-06 07:01:00       dairy
                       2019-09-06 07:02:00       dairy
                       2019-09-06 07:03:00       dairy
                       2019-09-06 07:04:00      spices
                       2019-09-06 07:05:00    checkout
             Monday    2019-09-02 07:03:00       dairy
                       2019-09-02 07:04:00       dairy
                       2019-09-02 07:05:00    checkout
             Thursday  2019-09-05 07:00:00      drinks
Name: location, dtype: object

In [7]:
frame_group.reset_index()

,customer_no,day,timestamp,location
0,1,Friday,2019-09-06 07:00:00,dairy
1,1,Friday,2019-09-06 07:01:00,dairy
2,1,Friday,2019-09-06 07:02:00,dairy
3,1,Friday,2019-09-06 07:03:00,dairy
4,1,Friday,2019-09-06 07:04:00,spices
...,...,...,...,...
53898,1533,Thursday,2019-09-05 21:48:00,dairy
53899,1534,Thursday,2019-09-05 21:48:00,fruit
53900,1534,Thursday,2019-09-05 21:49:00,checkout
53901,1535,Thursday,2019-09-05 21:48:00,spices


In [9]:
frame_trans = frame.reset_index().groupby(['customer_no', 'day'])[['timestamp', 'location']].agg(lambda x: list(x))

In [10]:
# creating matrix with states before and states after
frame_transition=pd.DataFrame(columns=['before','after'])
index = 0
frame_transition

,before,after


In [11]:
for i in frame_trans.iterrows():
    customer_no = i[0][0]
    day = i[0][1]
    timestamp = i[1][0]
    location = i[1][1]
    #print(sections)
    #location.insert(0,'entrance')
    #timestamp.insert(0,timestamp[0])
    #print(sections)
    #print(time_stamp)
    for j in range(len(location)):
        if (j+1)<len(location):
            frame_transition.loc[index]=(location[j],location[j+1])
            index += 1

In [12]:
frame_transition

,before,after
0,dairy,spices
1,spices,checkout
2,dairy,checkout
3,drinks,checkout
4,fruit,drinks
...,...,...
17427,spices,drinks
17428,drinks,checkout
17429,drinks,checkout
17430,fruit,checkout


In [24]:
#creating transition probability matrix
trans_prob_matrix = pd.crosstab(frame_transition['before'], frame_transition['after'], normalize=0)
trans_prob_matrix.loc['checkout'] = [1,0,0,0,0]

In [22]:
# ordering the columns inside
trans_prob_matrix = pd.DataFrame(trans_prob_matrix,columns=['dairy','drinks','fruit','spices','checkout'],index=['dairy','drinks','fruit','spices','checkout'])
#print(transition_probability_matrix)

In [23]:
trans_prob_matrix

,dairy,drinks,fruit,spices,checkout
dairy,0.000000,0.223151,0.189925,0.195713,0.391211
drinks,0.027159,0.000000,0.219062,0.216756,0.537023
fruit,0.238319,0.136266,0.000000,0.125904,0.499511
spices,0.323553,0.273140,0.152307,0.000000,0.251000
checkout,0.000000,0.000000,0.000000,0.000000,1.000000


In [25]:
assert all(trans_prob_matrix.sum(axis=1) > 0.999)

In [26]:
trans_prob_matrix.to_csv('trans_matrix.csv')